# Project #1


## Import Dependencies

https://pypi.org/project/FlavorTexts/

In [ ]:
pip install FlavorTexts

In [ ]:
from FlavorTexts import FlavorTexts
from tabulate import tabulate
import random 
import json

## Code For Class

In [ ]:
#List of Possible Weapons to be Assigned
item_options = ['Sword', 'Staff', 'Bow', 'Knife', 'Mace', 'Stick', 'Tome', 'Sigil', 'Gaulet'
                , 'Flail']

#List of Possible Pieces of Armor to be Assigned
armorOptions = ['Chestplate', 'Helmet']

#Property Bias for Certain Types of Armor
armorPropertyBias = {'Chestplate': {'protection' : 10, 'attack_spd': 0.5, 'durability': 5}
                     , 'Helmet': {'protection' : 10, 'attack_spd': 1, 'durability': 5}}

#Options for User Selectable Roles
role_options = {'A': 'Mage', 'B': 'Bishop', 'C': 'Knight'}

#Min/Max Item 
itemAttributesLim = [0.0, 2.0]
maxItemNum = 20

#Instanciates the random flavor text generator
generator = FlavorTexts(patreon=False)

class Character(object):
    """Create Character"""
    def __init__(self, name, role, items = {}, equipped = {}, build = {}):
        """Initialize name, role, items, equipped, build attributes."""
        self.name = name
        self.role = role
        self.items = items
        self.equipped = equipped
        self.build = build
        if self.role == 'Mage':
            self.build['magic_dmg'] = 1.5
            self.build['protection'] = 0.5
            self.build['melee_dmg'] = 0.65
            self.build['attack_spd'] = 1.5
        
        if self.role == 'Bishop':
            self.build['magic_dmg'] = 0.25
            self.build['protection'] = 2
            self.build['melee_dmg'] = 1.25
            self.build['attack_spd'] = 1
            
        if self.role == 'Knight':
            self.build['magic_dmg'] = 0.02
            self.build['protection'] = 1
            self.build['melee_dmg'] = 1.5
            self.build['attack_spd'] = 0.75
        
        #Randomly generates items for character if new character
        if len(items) == 0:
            for i in range(10):    
                self.generate_weapons()
            for i in range(2):    
                self.generate_armor()
    
    #Randomly generating armor
    def generate_armor(self):
        if len(self.items) == maxItemNum:
            print("Inventory Full")
        else:    
            while(True):
                #Randomly pick a unique id for new item
                uuid_item = random.randrange(0,maxItemNum)
                if(uuid_item not in self.items):
                    armorPiece = armorOptions[random.randrange(0, len(armorOptions))]
                    self.items[uuid_item] = {'name': armorPiece,
                    'protection': round(armorPropertyBias[armorPiece]['protection']*
                                        random.uniform(itemAttributesLim[0], itemAttributesLim[1]), 2),
                    'attack_spd': round(armorPropertyBias[armorPiece]['attack_spd']*
                                        random.uniform(itemAttributesLim[0], itemAttributesLim[1]), 2),
                    'durability': round(armorPropertyBias[armorPiece]['durability']*
                                        random.uniform(itemAttributesLim[0], itemAttributesLim[1]), 2),
                    'descript': ''.join(generator.generateText(1)), 'item_type': 'armor'}
                    break
    
    #Randomly generating weapons
    def generate_weapons(self):
        if len(self.items) == maxItemNum:
            print("Inventory Full")
        else:    
           # while(True):
                #Randomly pick a unique id for new item
                uuid_item = random.randrange(0,maxItemNum)
                if uuid_item not in self.items:
                    self.items[uuid_item] = {'name': item_options[random.randrange(0, len(item_options))],
                    'magic_dmg': round(random.uniform(itemAttributesLim[0], itemAttributesLim[1]), 2),
                    'melee_dmg': round(random.uniform(itemAttributesLim[0], itemAttributesLim[1]), 2),
                    'attack_spd': round(random.uniform(itemAttributesLim[0], itemAttributesLim[1]), 2),
                    'descript': ''.join(generator.generateText(1)), 'item_type': 'weapon'}
                   # break
                    
    #Function to nicely display the character's inventory can select to see what is equipped                
    def dispInventory(self, itemSet = 'inventory'):
        if itemSet == 'inventory':
            target = self.items
        if itemSet == 'equipped':
            target = self.equipped
        
        outputArray = []
        #Arranging data in inventory dictionrary into an array for tabulate function
        for i in target:
            
            if target[i]['item_type'] == 'weapon': 
                outputArray.append([i, target[i]['name'], target[i]['item_type']
                                      , target[i]['descript']
                                      , target[i]['magic_dmg'], target[i]['melee_dmg']
                                      , target[i]['attack_spd'], 'Na', 'Na'])
            if target[i]['item_type'] == 'armor': 
                outputArray.append([i, target[i]['name'], target[i]['item_type']
                                      , target[i]['descript']
                                      , 'Na', 'Na', target[i]['attack_spd']
                                      , target[i]['protection'], target[i]['durability']])
            
        print(tabulate(outputArray, headers=['ID#', 'Name', 'Type', 'descript', 'Magic'
                                            , 'Melee', 'Atk Spd', 'Prot', 'Dura']
                                            , numalign="center", stralign="left"))
    #Moves item from inventory dictionary to equipped dictionary
    def equipItems(self, equipList):
        for i in equipList:
            if i in self.items:
                self.equipped[i] = self.items.pop(i)
            else:
                print("Item not in inventory")

    #Moves item from equipped dictionary to inventory dictionary            
    def dequipItems(self, dequipList):
        for i in dequipList:
            if i in self.equipped:
                self.items[i] = self.equipped.pop(i)
            else:
                print("Item not equipped")
                
    #Calculate the statistics of the player with the current items equipped      
    def dispStats(self):
        properties = ['magic_dmg', 'melee_dmg', 'attack_spd', 'protection']
        charProps = [self.build['magic_dmg'], self.build['melee_dmg'], self.build['attack_spd']
                    , self.build['protection']]

        totalBuffs = []
        index = 0
        for props in properties:
            totalBuffs.append(self.build[props])
            for items in self.equipped:
                if self.equipped[items]['item_type'] == 'armor':
                    if props == 'magic_dmg' or props == 'melee_dmg':  
                        totalBuffs[index] = totalBuffs[index]
                    else:    
                        totalBuffs[index] = (round(totalBuffs[index]*(1+self.equipped[items][props])
                                                                      , 2))
                if self.equipped[items]['item_type'] == 'weapon':
                    if props == 'protection': 
                        totalBuffs[index] = totalBuffs[index]
                    else:    
                        totalBuffs[index] = (round(totalBuffs[index]*(1+self.equipped[items][props])
                                                  , 2))
            index += 1
        
        #Rearrange data for tabulate funciton to display properly
        outputArray = []
        for i in range(4):
            outputArray.append([properties[i], charProps[i], totalBuffs[i]])
            
        print(tabulate(outputArray, headers=['Property', 'Before Buffs', 'After Buffs']
                                            , numalign="center", stralign="left"))
        

## Other Functions

In [ ]:

#Contains the code to load/save/run the game

filename = 'game.json'

    
    #Function to load saved character from file, is passed json data
    
def loadSavedChar(file):
        print("Load a player:")
        num = 1

        #outputting characters in the saved file
        for i in file:
            print('#' + str(num)+': ' + file[i]['name'])
            num += 1

        selection = input("Type in your character name or type 'n' to make a new character")

        if selection.upper() == 'N':
            newCharacter = (saveChar(newCharCreation()))
            file[newCharacter['name']] = newCharacter
            with open(filename, 'w') as f:
                json.dump(file, f)
            print("Thank you for playing!")

        else:
            loadplayer = Character(file[selection]['name'], file[selection]['role'], file[selection]['items'], file[i]['equipped'],
                              file[selection]['build'])
            print("\nLoading: " + selection + " Role: " + loadplayer.role)
            print("\nPlayer Stats")
            loadplayer.dispStats()
            print("\nEquipped Items: ")
            loadplayer.dispInventory("equipped")
            print("\nInventory: ")
            loadplayer.dispInventory()

            print("\nThank you for playing again! Bye!")  
def saveChar(character):
    data = {'name': character.name, 'role': character.role, 'items': character.items
                            , 'equipped': character.equipped, 'build': character.build}
    return data
    
def newCharCreation():
    nameInput = input("Hi there! I see you are new! What is your name? ")
    print("Here are the classes that exist: " + str(role_options))
    roleInput = input("What role do you want to be? ")
    print("\nLooks like you picked " + role_options[roleInput.upper()])

    player = Character(nameInput, role_options[roleInput.upper()])

    print("\nHere is an overview of your Inventory:")
    player.dispInventory()

    print("\nWhat items would you like to equip? You can only equip 1 weapon and 2 pieces of armor: ")
    equipitems = []
    for i in range(0, 3):
        element = int(input("Item Selection #" + str(i+1) + ": "))
        equipitems.append(element)

    print("\nEquipping Items...")    
    player.equipItems(equipitems)
    print("\nHere are the items that are equipped: ")
    player.dispInventory('equipped')

    print("\nHere is an overview of your Inventory:")
    player.dispInventory()

    print("\nHere is an overview of your Stats:")
    player.dispStats()
    
    print("\nAlright! Now Saving Data!")
    return player
def startUpGame():
    
    #Tried to load game file but if it doesn't exist, start from scratch
    try:
        with open(filename, 'r') as f:
            file = json.load(f)
    except FileNotFoundError:
        newCharacter = (saveChar(newCharCreation()))
        with open(filename, 'w') as f:
            json.dump({newCharacter['name']:newCharacter}, f)
        print("Thank you for playing!")
        
    else:
        print("Welcome back!!!")
        loadSavedChar(file)

## Code to Run to Play Game

In [ ]:
#file
startUpGame()



In [ ]:
while(True): 
    answer = input("Would you like to play a game? (yes/no) ").lower().strip()
    if answer == "yes":

        while(True):
            while(True):
                player_type = input("Choose your player type: [W]Wizard or [K]Knight: ").lower().strip()

                if player_type == "w":
                    print("You've chosen Wizard! ")
                    break
                elif player_type == 'k':
                    print("You've chosen Knight! ")
                    break
                else:
                    print("Invalid input. Please choose either W or K.")
                
            while(True):
                answer = input("You reach a cross roads, would you like to go left or right? ").lower().strip()
                if answer == "left":
                    answer = input("You encounter a lion, would you like to run or attack? ").lower().strip()

                    if answer == "attack":
                        print("You are not armed, you were eaten alive! ")
                        print("Game over!")
                        break;

                    elif answer == "run":
                        print("Good choice, you got away safely.")
                        break

                elif answer.lower().strip() == "right":
                    print("You fell in a pit of snakes!")
                    print("Game over!")
                    break;

                else:
                    print("huh?...")
                    print("Invalid input!")

    elif answer == "no":
        print("Have a great day!")
        break;

    else:
        print("Invalid choice.")